In [1]:
from netCDF4 import Dataset
import os

import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata


import matplotlib.tri as tri

## Merging Turbidity data from BAWs Untrim data set with Johannes' SCHISM grid



### BAW data

In [2]:
baw = Dataset('/scratch/local1/baw/f.AZHel_FT_REF.2D.cut.nc', 'r')

In [3]:
_ = [print(item) for item in baw.variables.keys()]

Combined_Mesh2_and_SubMesh2
Mesh2
Mesh2_Schwebstoffklassen_2d
Mesh2_crs
Mesh2_edge_Durchflussflaeche_2d
Mesh2_edge_Gesamtwassertiefe_an_Kanten_2d
Mesh2_edge_Stroemungsgeschwindigkeit_x_R_2d
Mesh2_edge_Stroemungsgeschwindigkeit_y_R_2d
Mesh2_edge_bc
Mesh2_edge_faces
Mesh2_edge_hor_Wassertransport_Kantenflaeche_2d
Mesh2_edge_hor_adv_Salztransport_Kantenflaeche_2d
Mesh2_edge_hor_adv_Schwebstofftransport_Kantenfl_2d
Mesh2_edge_hor_adv_Waermetransport_Kantenflaeche_2d
Mesh2_edge_hor_dif_Salztransport_Kantenflaeche_2d
Mesh2_edge_hor_dif_Schwebstofftransport_Kantenfl_2d
Mesh2_edge_hor_dif_Waermetransport_Kantenflaeche_2d
Mesh2_edge_lat
Mesh2_edge_lat_bnd
Mesh2_edge_lon
Mesh2_edge_lon_bnd
Mesh2_edge_mit_hor_durchstroemte_Kantenflaeche_2d
Mesh2_edge_mittlere_nasse_Laenge_an_Kanten_2d
Mesh2_edge_nasse_Laenge_an_Kanten_2d
Mesh2_edge_nodes
Mesh2_edge_skalare_Stroemungsgeschwindigkeit_2d
Mesh2_edge_x
Mesh2_edge_x_bnd
Mesh2_edge_y
Mesh2_edge_y_bnd
Mesh2_edge_z_edge_2d
Mesh2_edge_z_edge_bnd_2d
Mesh2_f

In [4]:
baw.variables['Mesh2_Schwebstoffklassen_2d']

<class 'netCDF4._netCDF4.Variable'>
|S1 Mesh2_Schwebstoffklassen_2d(nMesh2_suspension_classes, nMesh2_class_names_strlen)
    long_name: Schwebstoffklassen
    name_id: 1704
unlimited dimensions: 
current shape = (6, 80)
filling on, default _FillValue of   used

In [5]:
[print(''.join([byte.decode('utf-8') for byte in baw.variables['Mesh2_Schwebstoffklassen_2d'][ii].data])) for ii in range(6)]

Summe aller Fraktionen                                                          
Very_Fine_Sand                                                                  
Coarse_Silt                                                                     
Medium_Silt                                                                     
Fine_Silt                                                                       
Very_Fine_Silt                                                                  


[None, None, None, None, None, None]

In [6]:
baw_face_x = baw.variables['Mesh2_face_x'][:]
baw_face_y = baw.variables['Mesh2_face_y'][:]

baw_spm = baw.variables['Mesh2_face_Schwebstoffgehalt_2d']

In [7]:
spm = {
    'sum_of_all_classes': baw_spm[:, 0, 0],
    'very_fine_silt': baw_spm[:, 1, 0],
    'fine_silt': baw_spm[:, 2, 0],
    'medium_silt': baw_spm[:, 3, 0],
    'coarse_silt': baw_spm[:, 4, 0],
    'very_fine_sand': baw_spm[:, 5, 0]
}

### HZG data

In [8]:
hzg = Dataset('/scratch/local1/output/22_11_01_depth_losses_v10/22_11_01_depth_losses_v10_grid.nc', 'r')

In [9]:
_ = [print(item) for item in hzg.variables.keys()]

x
triangles
triangle_area
adjacency
node_type
is_boundary_triangle
water_depth


In [10]:

hzg_node_x = hzg.variables['x'][:,0]
hzg_node_y = hzg.variables['x'][:,1]

hzg_tri = hzg.variables['triangles'][:]

## Creating a copy of the HZG dataset with the interpolated SPM values

Created a copy manually with rsync

### Interpolating and merging BAW data onto HZG grid

#### griddata interpolation

In [11]:
!ls path_to_dir2

ls: cannot access 'path_to_dir2': No such file or directory


In [12]:
# list files in path_to_dir2
path_to_dir2 = '/scratch/local1/hzg2'
files = os.listdir(path_to_dir2)
files

['schout_272.nc',
 'schout_111.nc',
 'schout_199.nc',
 'schout_197.nc',
 'schout_156.nc',
 'schout_171.nc',
 'schout_63.nc',
 'schout_116.nc',
 'schout_252.nc',
 'schout_113.nc',
 'schout_294.nc',
 'schout_140.nc',
 'schout_73.nc',
 'schout_91.nc',
 'schout_350.nc',
 'schout_185.nc',
 'schout_139.nc',
 'schout_95.nc',
 'schout_179.nc',
 'schout_158.nc',
 'schout_92.nc',
 'schout_354.nc',
 'schout_321.nc',
 'schout_285.nc',
 'schout_60.nc',
 'schout_2.nc',
 'schout_284.nc',
 'schout_307.nc',
 'schout_226.nc',
 'schout_24.nc',
 'schout_65.nc',
 'schout_250.nc',
 'schout_155.nc',
 'schout_127.nc',
 'schout_132.nc',
 'schout_175.nc',
 'schout_47.nc',
 'schout_86.nc',
 'schout_67.nc',
 'schout_149.nc',
 'schout_174.nc',
 'schout_187.nc',
 'schout_298.nc',
 'schout_341.nc',
 'schout_55.nc',
 'schout_52.nc',
 'schout_277.nc',
 'schout_33.nc',
 'schout_159.nc',
 'schout_44.nc',
 'schout_193.nc',
 'schout_82.nc',
 'schout_133.nc',
 'schout_177.nc',
 'schout_142.nc',
 'schout_362.nc',
 'schout_3

In [14]:
for fraction in spm.keys():
    print(fraction)

    # Create source points from baw data
    source_points = np.array([baw_face_x, baw_face_y]).T

    # Create target grid points from hzg data
    target_points = np.array([hzg_node_x, hzg_node_y]).T

    source_data = spm[fraction]
    source_data = np.mean(source_data, axis=0)

    # Interpolate baw data onto hzg grid
    interpolated_spm = griddata(source_points, source_data, target_points, method='linear')
    # Note: The 'method' argument can be 'linear', 'nearest', or 'cubic'.

    # set 0- where nan
    interpolated_spm[np.isnan(interpolated_spm)] = 0
   

    path_to_dir2 = '/scratch/local1/hzg2'

    for root, dirs, files in os.walk(path_to_dir2):
        n_files = len(files)
        for ii,momo in enumerate(sorted(files)):
            if momo.endswith('.nc'):
                print(momo, f'( {ii+1} / {n_files} )')
                with Dataset(os.path.join(root, momo), 'a') as ds:
                    # Ensure the dimension exists
                    if 'nSCHISM_hgrid_node' not in ds.dimensions:
                        raise ValueError("The dimension 'nSCHISM_hgrid_node' does not exist in the file.")

                    # if not, create it
                    if f'spm_{fraction}' not in ds.variables:
                        spm_var = ds.createVariable(f'spm_{fraction}', 'f4', ('nSCHISM_hgrid_node',))

                    # get the variable
                    spm_var = ds.variables[f'spm_{fraction}']

                    # Assign the turbidity data to the variable
                    spm_var[:] = interpolated_spm

                    # If needed, set attributes for the turbidity variable
                    spm_var.units = 'kg/l'  # Adjust this as necessary
                    spm_var.long_name = f'Suspended sediment concentration of {fraction}'
                    # add description
                    spm_var.description = 'Suspended sediment concentration based on BAWs UnTrim model using the Dietrich model received from Arne Hammrich. Average of 3D data over time and depth'


    

sum_of_all_classes
schout_1.nc ( 2 / 367 )
schout_10.nc ( 3 / 367 )
schout_100.nc ( 4 / 367 )
schout_101.nc ( 5 / 367 )
schout_102.nc ( 6 / 367 )
schout_103.nc ( 7 / 367 )
schout_104.nc ( 8 / 367 )
schout_105.nc ( 9 / 367 )
schout_106.nc ( 10 / 367 )
schout_107.nc ( 11 / 367 )
schout_108.nc ( 12 / 367 )
schout_109.nc ( 13 / 367 )
schout_11.nc ( 14 / 367 )
schout_110.nc ( 15 / 367 )
schout_111.nc ( 16 / 367 )
schout_112.nc ( 17 / 367 )
schout_113.nc ( 18 / 367 )
schout_114.nc ( 19 / 367 )
schout_115.nc ( 20 / 367 )
schout_116.nc ( 21 / 367 )
schout_117.nc ( 22 / 367 )
schout_118.nc ( 23 / 367 )
schout_119.nc ( 24 / 367 )
schout_12.nc ( 25 / 367 )
schout_120.nc ( 26 / 367 )
schout_121.nc ( 27 / 367 )
schout_122.nc ( 28 / 367 )
schout_123.nc ( 29 / 367 )
schout_124.nc ( 30 / 367 )
schout_125.nc ( 31 / 367 )
schout_126.nc ( 32 / 367 )
schout_127.nc ( 33 / 367 )
schout_128.nc ( 34 / 367 )
schout_129.nc ( 35 / 367 )
schout_13.nc ( 36 / 367 )
schout_130.nc ( 37 / 367 )
schout_131.nc ( 38 / 36

In [15]:
df = Dataset('/scratch/local1/hzg2/schout_1.nc')
# print all variables with spm in name
_ = [print(item) for item in df.variables.keys() if 'spm' in item]
df.close()

spm_sum_of_all_classes
spm_very_fine_silt
spm_fine_silt
spm_medium_silt
spm_coarse_silt
spm_very_fine_sand


In [16]:
df = Dataset('/scratch/local1/hzg2/schout_1.nc')
# print all variables with spm in name
print(df.variables['spm_sum_of_all_classes'][:].shape)
df.close()

(32432,)
